In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from operator import add
import pandas as pd

In [3]:
sc = SparkContext("local[*]")

In [4]:
# https://udb.moe.edu.tw/Historical
# 3.學士班以下延修生數
# 9.修讀輔系人次
# 10.修讀雙主修人次

# 讀檔
raw_delay_data = sc.textFile("univ_delay.csv")
raw_minor_data = sc.textFile("univ_minor.csv")
raw_dual_data = sc.textFile("univ_dual.csv")

# raw_delay_data.take(5)
raw_minor_data.take(5)
# raw_dual_data.take(5)

[',學期,學校,,,學校名稱,系所代碼,系所名稱,學制,輔系人次',
 ',,設立別 ,學校類別,學校代碼,,,,,',
 '101,1,公立,一般大學,1,國立政治大學,140101,教育學系,博士班,0',
 '101,1,公立,一般大學,1,國立政治大學,140101,教育學系,碩士班(日間),0',
 '101,1,公立,一般大學,1,國立政治大學,140101,教育學系,學士班(日間),157']

In [5]:
# 分割欄位
split_delay = raw_delay_data.map(lambda data: data.split(','))
split_minor = raw_minor_data.map(lambda data: data.split(','))
split_dual = raw_dual_data.map(lambda data: data.split(','))

# split_delay.take(5)
split_minor.take(5)
# split_dual.take(5)

[['', '學期', '學校', '', '', '學校名稱', '系所代碼', '系所名稱', '學制', '輔系人次'],
 ['', '', '設立別 ', '學校類別', '學校代碼', '', '', '', '', ''],
 ['101', '1', '公立', '一般大學', '1', '國立政治大學', '140101', '教育學系', '博士班', '0'],
 ['101', '1', '公立', '一般大學', '1', '國立政治大學', '140101', '教育學系', '碩士班(日間)', '0'],
 ['101', '1', '公立', '一般大學', '1', '國立政治大學', '140101', '教育學系', '學士班(日間)', '157']]

In [6]:
# 淡江大學ID: 1005
# 資訊工程學系ID: 520114

# 只選取 淡江大學 學士班(日間) 的資料
def select_tku_csie(line):
    isBachelor = '學士班(日間)' in line[8]
    if line[4] == '1005' and isBachelor:
        return line

tku_delay = split_delay.filter(lambda line: select_tku_csie(line))
tku_minor = split_minor.filter(lambda line: select_tku_csie(line))
tku_dual = split_dual.filter(lambda line: select_tku_csie(line))

# tku_delay.take(3)
tku_minor.take(3)
# tku_dual.take(3)

[['101',
  '1',
  '私立',
  '一般大學',
  '1005',
  '淡江大學',
  '140801',
  '教育科技學系',
  '學士班(日間)',
  '10'],
 ['101',
  '1',
  '私立',
  '一般大學',
  '1005',
  '淡江大學',
  '220201',
  '中國文學學系',
  '學士班(日間)',
  '79'],
 ['101', '1', '私立', '一般大學', '1005', '淡江大學', '220309', '英文學系', '學士班(日間)', '82']]

In [7]:
# data: (學年度, 系所, 數量)
depart_delay = tku_delay.map(lambda row: (row[1], row[7], int(row[11])))
depart_delay.take(5)

[('101', '教育科技學系（日）', 9),
 ('101', '中國文學學系（日）', 34),
 ('101', '英文學系（日）', 28),
 ('101', '法國語文學系（日）', 28),
 ('101', '德國語文學系（日）', 11)]

In [8]:
# 把同年上下學期加總
# (key, value) = ((學年度, 系所), 數量)
temp_minor = tku_minor.map(lambda row: ((row[0], row[7]), int(row[9]))).reduceByKey(add)
temp_dual = tku_dual.map(lambda row: ((row[0], row[7]), int(row[9]))).reduceByKey(add)

# 整理成 data: (學年度, 系所, 數量) 的格式
depart_minor = temp_minor.map(lambda row: (row[0][0], row[0][1], row[1]))
depart_dual = temp_dual.map(lambda row: (row[0][0], row[0][1], row[1]))

depart_minor.take(5)
# dataset_dual.take(5)

[('101', '中國文學學系', 143),
 ('101', '英文學系', 144),
 ('101', '法國語文學系', 167),
 ('101', '俄國語文學系', 68),
 ('101', '英美語言文化學系', 2)]

In [9]:
# 將RDD轉換成Dataframe
spark = SparkSession(sc)
rdd = sc.parallelize(depart_delay.collect())
df_delay = rdd.toDF(["Year", "Depart", "Delay Count"]).orderBy("Year", "Depart")

rdd = sc.parallelize(depart_minor.collect())
df_minor = rdd.toDF(["Year", "Depart", "Minor Count"]).orderBy("Year", "Depart")

rdd = sc.parallelize(depart_dual.collect())
df_dual = rdd.toDF(["Year", "Depart", "Dual Count"]).orderBy("Year", "Depart")

# df_delay.take(5)
df_minor.take(5)
# df_dual.take(5)

[Row(Year='101', Depart='中國文學學系', Minor Count=143),
 Row(Year='101', Depart='企業管理學系', Minor Count=83),
 Row(Year='101', Depart='俄國語文學系', Minor Count=68),
 Row(Year='101', Depart='保險學系', Minor Count=72),
 Row(Year='101', Depart='全球政治經濟學系', Minor Count=0)]

In [10]:
# 合併所有Dataframe
tku_depart_data = df_delay.join(df_minor, ["Year", "Depart"], "full").join(df_dual, ["Year", "Depart"], "full").orderBy("Year", "Depart").toPandas()
tku_depart_data.fillna(0, inplace=True)
tku_depart_data.shape

(346, 5)

In [11]:
# 秀出前30筆資料
tku_depart_data.head(30)

,Year,Depart,Delay Count,Minor Count,Dual Count
0,101,中國文學學系,0.0,143.0,42.0
1,101,中國文學學系（日）,34.0,0.0,0.0
2,101,企業管理學系,0.0,83.0,14.0
3,101,企業管理學系（日）,36.0,0.0,0.0
4,101,俄國語文學系,0.0,68.0,20.0
5,101,俄國語文學系（日）,6.0,0.0,0.0
6,101,保險學系,0.0,72.0,18.0
7,101,保險學系（日）,15.0,0.0,0.0
8,101,全球政治經濟學系,0.0,0.0,0.0
9,101,全球政治經濟學系（日）,2.0,0.0,0.0


In [ ]:
'''
從上表的資料可以發現一般容易延畢的科系都不會有修輔系或雙主修的人
但是輔系、雙主修又的確是延畢的重要因素

例如：
    英文系的同學雙主修資工學位，假如他順利從英文系畢業，但在資工系需要延畢，
    紀錄上只會有英文系雙主修與資工系延畢
    
從單個系所的資料看不出兩者之間的關係，因此，我們需要從全校的整體數據進行統整。
'''